In [1]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter hide_code
{% do SectionField(
  name='primary',
  title='C2M2 FAIR Assessment',
  subtitle='Assessing c2m2 datapackages for FAIRness',
) %}

{% set file = FileField(
  name='file',
  label='A zipped [C2M2 Datapackage](https://docs.nih-cfde.org/en/latest/c2m2/draft-C2M2_specification/)',
  help='Provide your zipped c2m2 datapackage',
  examples={'example.zip': url_for('static', path='example.zip')},
  default='static/example.zip',
  section='primary',
) %}

# C2M2 Assessment

We perform a file-centric FAIR Assessment on all files defined in a [C2M2 datapackage](https://docs.nih-cfde.org/en/latest/c2m2/draft-C2M2_specification/) according to the [C2M2 Rubric](https://fairshake.cloud/rubric/36); descriptions of each metric and how we assess them are provided below, along with the actual code to perform the assesssment.

In [ ]:
import os
import re
import glob
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from textwrap import dedent
from IPython.display import display, Markdown

In [ ]:
def one_and_only(it):
  ''' Select one and only item from an iterable, otherwise throw an exception.
  '''
  it = iter(it)
  ret = next(it)
  try:
    next(it)
    raise Exception('Expected one')
  except StopIteration:
    return ret

def deep_find(root, file):
  ''' Helper for finding a filename in a potentially deep directory
  '''
  return set.union(
    set(glob.glob(os.path.join(root, file))),
    set(glob.glob(os.path.join(root, '**', file))),
  )

def fetch_cache(url, filename, cachedir='.cached'):
  ''' Download a {file} from a {url} if it hasn't already been downloaded, storing it in {cachedir}.
  '''
  import os, urllib.request
  os.makedirs(cachedir, exist_ok=True)
  if not os.path.exists(os.path.join(cachedir, filename)):
    urllib.request.urlretrieve(url, filename=os.path.join(cachedir, filename))
  return os.path.join(cachedir, filename)

def url_join(*args):
  ''' Join urls by slashes, not worrying about duplicated trailing slashes
  '''
  return '/'.join([arg.rstrip('/') for arg in args[:-1]]+[args[-1]])

def filter_empty(val):
  ''' Attempt to catch some actual null values that aren't really null.
  '''
  return [
    v
    for v in val
    if v is not None and (
      type(v) != str or v.strip().lower() not in {
        '-',
        '-666',
        '',
        'empty',
        'n/a',
        'na',
        'nan',
        'nil',
        'none',
        'not defined',
        'null',
        'undef',
        'undefined',
      }
    )
  ]

newline = '\n'

## Step 1. Load DERIVA compatible client from URL or datapackage

Given a datapackage, access it through DERIVA-compatible client. This client package (`deriva_compat.py`) permits accesisng offline datapackages in the same way that the online DERIVA client operates, thus the assessment can be performed online or offline.

In [ ]:
%%appyter code_exec
import zipfile

file = {{ file }}
basename, ext = os.path.splitext(file)
assert ext == '.zip', 'Expected .zip file'
directory = basename

with zipfile.ZipFile(file, 'r') as z:
  z.extractall(directory)

In [ ]:
from deriva_compat import create_offline_client

# sometimes zip files zip the leading directory, which may be named anything,
#  deep_find lets us locate the datapackage wherever it is.
CFDE = create_offline_client(deep_find(directory, 'datapackage.json'))

## Step 2. Compute and report global metrics about the datapackage

In [ ]:
total_files = CFDE.tables['file'].count()
total_collections = CFDE.tables['collection'].count()
total_biosamples = CFDE.tables['biosample'].count()
total_subjects = CFDE.tables['subject'].count()
total_projects = CFDE.tables['project'].count()

In [ ]:
display(Markdown('### Table Counts'))
pd.Series({
  name: table.count()
  for name, table in CFDE.tables.items()
}).to_frame('Table Counts')

In [ ]:
display(Markdown('### Table Metadata Coverage'))
coverage = pd.DataFrame(
  dict(
    table=table_name,
    coverage=len(list(filter_empty(entity.values()))) / len(table.column_definitions.keys()),
  )
  for table_name, table in CFDE.tables.items()
  for entity in table.entities()
).groupby('table')['coverage'].describe()
coverage

## Step 3. Compute and report coverage metrics about the datapackage

We use a python decorator for registering each metric into the rubric. This lets us define each metric in its own cell with its description and code to assert it. Because of the nature of these assertions, assessments are performed as metrics are registered. This assessment is datapackage-centric rather than file centric.

This paradigm can be used for any rubric allowing assessment code to remain the same even with changing metrics, furthermore this is compatible with [FAIRshake](https://fairshake.cloud/) assessments, adopting FAIRshake metric identifiers allowing the results to be easily registered with FAIRshake.

In [ ]:
rubric = {
  '@id': 36,
  'name': 'NIH CFDE Interoperability',
  'description': 'This rubric identifies aspects of the metadata models which promote interoperable dataset querying and filtering',
  'metrics': {},
}
answers = []

def _register_metric_answer(schema):
  global metrics
  global answers
  def wrapper(func):
    rubric['metrics'][schema['@id']] = dict(schema, func=func)
    answer = dict(func(), metric=schema['@id'])
    answers.append(answer)
    link = '' if schema['@id'] < 0 else f" ([{schema['@id']}](https://fairshake.cloud/metric/{schema['@id']}))"
    display(Markdown(dedent(f'''
        ### Metric{link}: {schema['name']}
        **{schema['description']}**

        {schema['detail']}

        #### Results
    ''') + dedent(answer['comment'])))
  setattr(wrapper, '__name__', schema['name'])
  return wrapper

In [ ]:
@_register_metric_answer({
  # standardized metadata format (107), machine readable metadata (106)
  # metadata license (117) (c2m2 ?)
  '@id': 106,
  'name': 'Metadata conformance',
  'description': 'The metadata properly conforms with the CFDE perscribed metadata model specification',
  'detail': '''The average metadata coverage of all tables''',
  'principle': 'Findable',
})
def _():
  value = coverage['mean'].mean()
  return {
    'value': value,
    'comment': f'{value*100:.3f}%.\n\nSee metadata coverage for more info',
  }

In [ ]:
# TODO: Help us identify other acceptible globally unique persistent and valid identifiers
@_register_metric_answer({
  # Persistent identifier (105)
  '@id': 104,
  'name': 'Persistent identifier',
  'description': 'Globally unique, persistent, and valid identifiers (preferrably DOIs) are present for the dataset',
  'detail': '''We check that the persistent id that are present are DOIs.''',
  'principle': 'Findable',
})
def _():
  total_qualified_persistent_ids = sum(
    1 if file.get('persistent_id') and re.match(r'^https?://[^/]+\.doi\.org/.+$', file['persistent_id']) else 0
    for file in CFDE.tables['file'].entities()
  )
  value = total_qualified_persistent_ids / total_files
  return {
    'value': value,
    'comment': f'{value*100:.3f}% ({total_qualified_persistent_ids} / {total_files})',
  }

In [ ]:
@_register_metric_answer({
  '@id': -1,
  'name': 'ratio files are associated with data type term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_data_type = CFDE.tables['data_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].data_type == CFDE.tables['data_type'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_data_type / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_data_type} / {total_files})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -2,
  'name': 'ratio files are associated with file format term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_file_format = CFDE.tables['file_format'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].file_format == CFDE.tables['file_format'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_file_format / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_file_format} / {total_files})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -3,
  'name': 'ratio files are associated with assaytype term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_assay_type = CFDE.tables['assay_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].assay_type == CFDE.tables['assay_type'].id
    )) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_assay_type / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_assay_type} / {total_files})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -4,
  'name': 'ratio files are associate with anatomy term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_biosample'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_biosample'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_anatomy / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_anatomy} / {total_files})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -5,
  'name': 'ratio files are associated with a biosample',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_biosample = CFDE.tables['biosample'] \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_biosample'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_biosample'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_biosample / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_biosample} / {total_files})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -6,
  'name': 'ratio files are associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_subject'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_subject'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_subject / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_subject} / {total_files})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -7,
  'name': 'ratio files are associated with a subject_role_taxonomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_associated_with_subject_role = CFDE.tables['subject_role_taxonomy'] \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_describes_subject'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_describes_subject'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_associated_with_subject_role / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_associated_with_subject_role} / {total_files})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -8,
  'name': 'ratio biosamples are associated with a species term (NCBI Taxon)',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_ncbi_taxon = CFDE.tables['ncbi_taxonomy'] \
    .link(CFDE.tables['subject_role_taxonomy'], on=(
      CFDE.tables['subject_role_taxonomy'].taxonomy_id == CFDE.tables['ncbi_taxonomy'].id
    )) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_from_subject'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_from_subject'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_associated_with_ncbi_taxon / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_associated_with_ncbi_taxon} / {total_biosamples})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -9,
  'name': 'ratio biosamples are associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].subject_id_namespace == CFDE.tables['subject'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].subject_local_id == CFDE.tables['subject'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_from_subject'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_from_subject'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_associated_with_subject / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_associated_with_subject} / {total_biosamples})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -10,
  'name': 'ratio biosamples are associated with a file',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['file_describes_biosample'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['file_describes_biosample'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_associated_with_file / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_associated_with_file} / {total_biosamples})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -11,
  'name': 'ratio biosamples are associated with an anatomy term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_associated_with_anatomy / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_associated_with_anatomy} / {total_biosamples})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -12,
  'name': 'ratio biosamples are associated with an assay term',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_associated_with_assay = CFDE.tables['file'].filter(CFDE.tables['file'].assay_type != None)   .link(CFDE.tables['file_describes_biosample'], on=((
      CFDE.tables['file_describes_biosample'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_biosample'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['file_describes_biosample'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['file_describes_biosample'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_associated_with_assay / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_associated_with_assay} / {total_biosamples})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -13,
  'name': 'ratio subjects are associated with a taxonomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_taxonomy = CFDE.tables['ncbi_taxonomy'] \
    .link(CFDE.tables['subject_role_taxonomy'], on=(
      CFDE.tables['subject_role_taxonomy'].taxonomy_id == CFDE.tables['ncbi_taxonomy'].id
    )) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = total_subjects_associated_with_taxonomy / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_associated_with_taxonomy} / {total_subjects})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -14,
  'name': 'ratio subjects have subject granularity',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_granularity = CFDE.tables['subject'].filter(
    (CFDE.tables['subject'].granularity != None) & (CFDE.tables['subject'].granularity != '')
  ).count()
  value = total_subjects_associated_with_granularity / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_associated_with_granularity} / {total_subjects})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -15,
  'name': 'ratio subjects have taxonomic role',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_role_taxonomy = CFDE.tables['subject_role_taxonomy'] \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_role_taxonomy'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_role_taxonomy'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = total_subjects_associated_with_role_taxonomy / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_associated_with_role_taxonomy} / {total_subjects})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -16,
  'name': 'ratio subjects associated with a biosample',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_biosample = CFDE.tables['biosample'] \
    .link(CFDE.tables['biosample_from_subject'], on=((
      CFDE.tables['biosample_from_subject'].biosample_id_namespace == CFDE.tables['biosample'].id_namespace
    ) & (
      CFDE.tables['biosample_from_subject'].biosample_local_id == CFDE.tables['biosample'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['biosample_from_subject'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['biosample_from_subject'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = total_subjects_associated_with_biosample / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_associated_with_biosample} / {total_subjects})",
  }

In [ ]:

@_register_metric_answer({
  '@id': -17,
  'name': 'ratio subjects associated with a file',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['file_describes_subject'], on=((
      CFDE.tables['file_describes_subject'].file_id_namespace == CFDE.tables['file'].id_namespace
    ) & (
      CFDE.tables['file_describes_subject'].file_local_id == CFDE.tables['file'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['file_describes_subject'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['file_describes_subject'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = total_subjects_associated_with_file / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_associated_with_file} / {total_subjects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -18,
  'name': 'IF there are collections: # of files that are not part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_files_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['file_in_collection'], on=((
      CFDE.tables['file_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['file_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['file'], on=((
      CFDE.tables['file'].id_namespace == CFDE.tables['file_in_collection'].file_id_namespace
    ) & (
      CFDE.tables['file'].local_id == CFDE.tables['file_in_collection'].file_local_id
    ))) \
    .groupby(CFDE.tables['file'].id_namespace, CFDE.tables['file'].local_id) \
    .count()
  value = total_files_not_in_collection / total_files
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_files_not_in_collection} / {total_files})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -19,
  'name': 'IF there are collections: # of subjects that are not part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_subjects_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['subject_in_collection'], on=((
      CFDE.tables['subject_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['subject_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['subject'], on=((
      CFDE.tables['subject'].id_namespace == CFDE.tables['subject_in_collection'].subject_id_namespace
    ) & (
      CFDE.tables['subject'].local_id == CFDE.tables['subject_in_collection'].subject_local_id
    ))) \
    .groupby(CFDE.tables['subject'].id_namespace, CFDE.tables['subject'].local_id) \
    .count()
  value = total_subjects_not_in_collection / total_subjects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_subjects_not_in_collection} / {total_subjects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -20,
  'name': 'IF there are collections: # of biosamples that are not part of a collection',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  total_biosamples_not_in_collection = CFDE.tables['collection'] \
    .link(CFDE.tables['biosample_in_collection'], on=((
      CFDE.tables['biosample_in_collection'].collection_id_namespace == CFDE.tables['collection'].id_namespace
    ) & (
      CFDE.tables['biosample_in_collection'].collection_local_id == CFDE.tables['collection'].local_id
    ))) \
    .link(CFDE.tables['biosample'], on=((
      CFDE.tables['biosample'].id_namespace == CFDE.tables['biosample_in_collection'].biosample_id_namespace
    ) & (
      CFDE.tables['biosample'].local_id == CFDE.tables['biosample_in_collection'].biosample_local_id
    ))) \
    .groupby(CFDE.tables['biosample'].id_namespace, CFDE.tables['biosample'].local_id) \
    .count()
  value = total_biosamples_not_in_collection / total_biosamples
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_biosamples_not_in_collection} / {total_biosamples})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -21,
  'name': 'Project associated with anatomy',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_anatomy = CFDE.tables['anatomy'] \
    .link(CFDE.tables['biosample'], on=(
      CFDE.tables['biosample'].anatomy == CFDE.tables['anatomy'].id
    )) \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['biosample'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['biosample'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = total_projects_associated_with_anatomy / total_projects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_projects_associated_with_anatomy} / {total_projects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -22,
  'name': 'Project associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_file = CFDE.tables['file'] \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['file'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['file'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = total_projects_associated_with_file / total_projects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_projects_associated_with_file} / {total_projects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -23,
  'name': 'Project associated with data types',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_data_type = CFDE.tables['data_type'] \
    .link(CFDE.tables['file'], on=(
      CFDE.tables['file'].data_type == CFDE.tables['data_type'].id
    )) \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['file'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['file'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = total_projects_associated_with_data_type / total_projects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_projects_associated_with_data_type} / {total_projects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -24,
  'name': 'Project associated with subjects',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # NOTE: does not include recursive projects
  total_projects_associated_with_subject = CFDE.tables['subject'] \
    .link(CFDE.tables['project'], on=((
      CFDE.tables['project'].id_namespace == CFDE.tables['subject'].project_id_namespace
    ) & (
      CFDE.tables['project'].local_id == CFDE.tables['subject'].project_local_id
    ))) \
    .groupby(CFDE.tables['project'].id_namespace, CFDE.tables['project'].local_id) \
    .count()
  value = total_projects_associated_with_subject / total_projects
  return {
    'value': value,
    'comment': f"{value*100:.3f}% ({total_projects_associated_with_subject} / {total_projects})",
  }

In [ ]:
@_register_metric_answer({
  '@id': -25,
  'name': 'list of any anatomy terms in anatomy.tsv NOT associated with biosamples',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # TODO: we can probably use an outer join for this..
  used_anatomy_terms = CFDE.tables['biosample'] \
      .link(CFDE.tables['anatomy'], on=(
          CFDE.tables['anatomy'].id == CFDE.tables['biosample'].anatomy
      )) \
      .groupby(CFDE.tables['anatomy'].id)
  used_anatomy_ids = {
      anatomy['id']
      for anatomy in used_anatomy_terms.entities()
  }
  unused_anatomy_terms = pd.DataFrame({
      anatomy['id']: anatomy
      for anatomy in CFDE.tables['anatomy'].entities()
      if anatomy['id'] not in used_anatomy_ids
  }).T
  total = len(used_anatomy_ids) + unused_anatomy_terms.shape[0]
  value = 1 - (unused_anatomy_terms.shape[0] / total)
  return {
    'value': value,
    'comment': f"{value*100:.3f}% used ({len(used_anatomy_ids)} / {total})\n\n{newline.join(f'''- {v['id']}''' for _, v in unused_anatomy_terms.iterrows())}",
  }

In [ ]:
@_register_metric_answer({
  '@id': -26,
  'name': 'list of any species terms in ncbi_taxonomy.tsv NOT associated with a subject',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  # TODO: we can probably use an outer join for this..
  used_taxonomy_terms = CFDE.tables['subject'] \
      .link(CFDE.tables['subject_role_taxonomy'], on=((
          CFDE.tables['subject_role_taxonomy'].subject_id_namespace == CFDE.tables['subject'].id_namespace
      ) & (
          CFDE.tables['subject_role_taxonomy'].subject_local_id == CFDE.tables['subject'].local_id
      ))) \
      .link(CFDE.tables['ncbi_taxonomy'], on=(
          CFDE.tables['ncbi_taxonomy'].id == CFDE.tables['subject_role_taxonomy'].taxonomy_id
      )) \
      .groupby(CFDE.tables['ncbi_taxonomy'].id)
  used_taxonomy_ids = {
      taxonomy['id']
      for taxonomy in used_taxonomy_terms.entities()
  }
  unused_taxonomy_terms = pd.DataFrame({
      taxonomy['id']: taxonomy
      for taxonomy in CFDE.tables['ncbi_taxonomy'].entities()
      if taxonomy['id'] not in used_taxonomy_ids
  }).T
  total = len(used_taxonomy_ids) + unused_taxonomy_terms.shape[0]
  value = 1 - (unused_taxonomy_terms.shape[0] / total)
  return {
    'value': value,
    'comment': f"{value*100:.3f}% used ({len(used_taxonomy_ids)} / {total})\n\n{newline.join(f'''- {v['id']}''' for _, v in unused_taxonomy_terms.iterrows())}",
  }

In [ ]:
@_register_metric_answer({
  '@id': -27,
  'name': 'list of any assay terms in assay_type.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_assay_type_terms = CFDE.tables['file'] \
      .link(CFDE.tables['assay_type'], on=(
          CFDE.tables['assay_type'].id == CFDE.tables['file'].assay_type
      )) \
      .groupby(CFDE.tables['assay_type'].id)
  used_assay_type_ids = {
      assay_type['id']
      for assay_type in used_assay_type_terms.entities()
  }
  unused_assay_type_terms = pd.DataFrame({
      assay_type['id']: assay_type
      for assay_type in CFDE.tables['assay_type'].entities()
      if assay_type['id'] not in used_assay_type_ids
  }).T
  total = len(used_assay_type_ids) + unused_assay_type_terms.shape[0]
  value = 1 - (unused_assay_type_terms.shape[0] / total)
  return {
    'value': value,
    'comment': f"{value*100:.3f}% used ({len(used_assay_type_ids)} / {total})\n\n{newline.join(f'''- {v['id']}''' for _, v in unused_assay_type_terms.iterrows())}",
  }

In [ ]:
@_register_metric_answer({
  '@id': -28,
  'name': 'list of any format terms in file_format.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_file_format_terms = CFDE.tables['file'] \
      .link(CFDE.tables['file_format'], on=(
          CFDE.tables['file_format'].id == CFDE.tables['file'].file_format
      )) \
      .groupby(CFDE.tables['file_format'].id)
  used_file_format_ids = {
      file_format['id']
      for file_format in used_file_format_terms.entities()
  }
  unused_file_format_terms = pd.DataFrame({
      file_format['id']: file_format
      for file_format in CFDE.tables['file_format'].entities()
      if file_format['id'] not in used_file_format_ids
  }).T
  total = len(used_file_format_ids) + unused_file_format_terms.shape[0]
  value = 1 - (unused_file_format_terms.shape[0] / total)
  return {
    'value': value,
    'comment': f"{value*100:.3f}% used ({len(used_file_format_ids)} / {total})\n\n{newline.join(f'''- {v['id']}''' for _, v in unused_file_format_terms.iterrows())}",
  }

In [ ]:
@_register_metric_answer({
  '@id': -29,
  'name': 'list of any data type terms in data_type.tsv NOT associated with files',
  'description': '',
  'detail': '',
  'principle': '',
})
def _():
  used_data_type_terms = CFDE.tables['file'] \
      .link(CFDE.tables['data_type'], on=(
          CFDE.tables['data_type'].id == CFDE.tables['file'].data_type
      )) \
      .groupby(CFDE.tables['data_type'].id)
  used_data_type_ids = {
      data_type['id']
      for data_type in used_data_type_terms.entities()
  }
  unused_data_type_terms = pd.DataFrame({
      data_type['id']: data_type
      for data_type in CFDE.tables['data_type'].entities()
      if data_type['id'] not in used_data_type_ids
  }).T
  total = len(used_data_type_ids) + unused_data_type_terms.shape[0]
  value = 1 - (unused_data_type_terms.shape[0] / total)
  return {
    'value': value,
    'comment': f"{value*100:.3f}% used ({len(used_data_type_ids)} / {total})\n\n{newline.join(f'''- {v['id']}''' for _, v in unused_data_type_terms.iterrows())}",
  }

In [ ]:
@_register_metric_answer({
  '@id': 145,
  'name': 'Landing Page',
  'description': 'A landing page exists and is accessible for the identifiers',
  'detail': '''Checks to make sure the persistent_id is resolvable with a HEAD request. if it is not http/https it is assumed to be an identifiers.org-resolvable CURIE. note that this is still error prone, some identifier websites do not follow HTTP standards and may not report 404s with ids that aren't found.''',
  'principle': 'Findable',
})
def _():
  results = {}
  for file in CFDE.tables['file'].entities():
    persistent_id = file.get('persistent_id')
    if not persistent_id: continue
    if not re.match(r'^https?://', persistent_id):
      persistent_id = 'https://identifiers.org/{}'.format(persistent_id)
    #
    try:
      status_code = requests.head(persistent_id, headers={'User-Agent': None}).status_code
      if status_code >= 200 and status_code < 300:
        results[(file['id_namespace'], file['local_id'])] = 1
      elif status_code >= 300 and status_code < 399:
        results[(file['id_namespace'], file['local_id'])] = 0.5
      elif status_code >= 400:
        results[(file['id_namespace'], file['local_id'])] = 0.25
    except requests.exceptions.MissingSchema:
      results[(file['id_namespace'], file['local_id'])] = 0.0
  else:
    results[(file['id_namespace'], file['local_id'])] = 0.0
  results = pd.Series(results)
  display(results.describe())
  return {
    'value': results.mean(),
    'comment': f'{results.mean()*100:.3f}% availability',
  }

## Step 4. Review results

With the assessment complete, we're ready to review the results.

 ### Table 1. A simple look at the structure of the answers dataframe (joined with metrics for readability of metrics)

 - `name` this is the human readable name of the metric
 - `principle` this is the F.A.I.R category of the metric
 - `value` represents the quantitative value assigned to the given answer. It ranges between 0.0 and 1.0, 0.0 representing complete lack of *compliance* with a metric, and 1.0 representing complete satisfaction of a metric.
 - `comment` is a human-description describing why the `value` is what it is.

In [ ]:
df_metrics = pd.DataFrame(rubric['metrics']).T
df_answers = pd.merge(
    left=pd.DataFrame(answers), left_on='metric',
    right=df_metrics[['name', 'principle']], right_index=True,
)
df_answers[['name', 'principle', 'value', 'comment']].sort_values('value')

## Cleanup

No need to run this locally, but useful for appyter.

In [ ]:
import shutil
shutil.rmtree('.cached')
shutil.rmtree(directory)